In [81]:
import pandas as pd

## 등록된 제품 수

In [82]:
# 등록된 제품 수 by category, date
read_path = '../csv/products.csv'
df_product = pd.read_csv(read_path)

df_product.tail()

# status : 0: 판매중, 1: 예약됨, 2: 삭제, 3: 판매완료

,on_at,close_at,category_id,status,products
10846886,2018-07-19,2018-10-23,320090300.0,3,1
10846887,2018-05-20,2018-06-13,400081999.0,2,1
10846888,2018-09-27,2018-11-06,600700005.0,3,1
10846889,2018-12-25,2018-12-28,600600999.0,3,1
10846890,2018-08-01,2018-10-06,700700100.0,2,1


In [95]:
# 카테고리별 등록된 상품

df_on = df_product.drop(['close_at', 'status'], axis=1).rename(columns={'on_at': 'date_at'})

df_on['month_at'] = df_on['date_at'].str[:7] # day -> month
df_on['category_3'] = df_on['category_id'].astype(str).str[:3] # 1 depth category
df_on['category_6'] = df_on['category_id'].astype(str).str[:6] # 2 depth category

df_on_month = df_on.groupby(['month_at', 'category_3', 'category_6'], as_index=False).agg({'products': 'sum'})

df_on_month = df_on_month.rename(columns={'products': 'enrolled'})

df_on_month.tail()

,month_at,category_3,category_6,enrolled
4972,2019-05,910,910100,26
4973,2019-05,910,910200,14
4974,2019-05,910,910400,2
4975,2019-05,910,910800,1
4976,2019-05,999,999.0,34


In [ ]:
# 1-Depth Category

In [101]:
df_on_month_3 = df_on_month.groupby(['month_at', 'category_3'], as_index=False).agg({'enrolled': 'sum'})

df_on_month_3 = df_on_month_3.rename(columns={'category_3': 'category'})

df_on_month_3.tail()


,month_at,category,enrolled
535,2019-05,750,14
536,2019-05,800,47
537,2019-05,900,70
538,2019-05,910,43
539,2019-05,999,34


In [ ]:
# 2-Depth Category

In [109]:
df_on_month_6 = df_on_month.groupby(['month_at', 'category_6'], as_index=False).agg({'enrolled': 'sum'})

df_on_month_6 = df_on_month_6.rename(columns={'category_6': 'category'})

df_on_month_6.tail()


,month_at,category,enrolled
4972,2019-05,910100,26
4973,2019-05,910200,14
4974,2019-05,910400,2
4975,2019-05,910800,1
4976,2019-05,999.0,34


In [84]:
# save
save_path = '../csv/tmp/df_on_month.csv'
df_on_month.to_csv(save_path, index=False, mode='w', header=True)

## 카테고리 파일 읽기

In [85]:
# 카테고리

path = '../csv/categories.csv'

df_category = pd.read_csv(path)

df_category['category'] = df_category['category'].astype(str)

df_category = df_category[['category', 'name']]

df_category.tail()

,category,name
1219,240100030,서비스/미디어
1220,240100050,생산/기능직
1221,240100070,상담영업
1222,240100090,기타 업종
1223,240100110,단기알바


## 번프 완료 수

In [86]:
read_path = '../csv/bunp_category_since_1807.csv'
df_bunp = pd.read_csv(read_path)

df_bunp = df_bunp.rename(columns={'seller_pid': 'selled'})

df_bunp['category_3'] = df_bunp['category'].astype(str).str[:3] # 1 depth category
df_bunp['category_6'] = df_bunp['category'].astype(str).str[:6] # 2 depth category

df_bunp.tail()

,category,name,updated_at,selled,bunp_amount,category_3,category_6
3142303,240060050.0,IT/디자인 기타,2018-08-13,88103338,60000,240,240060
3142304,240100080.0,강사/교육,2018-08-20,86486421,13000,240,240100
3142305,240100080.0,강사/교육,2018-08-19,86486421,15000,240,240100
3142306,NaN,NaN,2019-01-28,95916529,254400,nan,nan
3142307,NaN,NaN,2018-12-23,93565603,87654,nan,nan


In [87]:
# 1-Depth Category

In [88]:
df_bunp_cate_3 = pd.merge(df_bunp, df_category[['category', 'name']], 
                        how='left', left_on=['category_3'], right_on=['category'])

df_bunp_cate_3['month_at'] = df_bunp_cate_3['updated_at'].str[:7]

df_bunp_cate_3 = df_bunp_cate_3.rename(columns={'category_y': 'category', 'name_y': 'category_name'})

df_bunp_cate_3 = df_bunp_cate_3.groupby(['month_at',
                                     'category',
                                     'category_name'], as_index=False).agg({'bunp_amount': 'sum',
                                                           'selled': 'count'})

df_bunp_cate_3.tail()

,month_at,category,category_name,bunp_amount,selled
174,2019-04,750,차량/오토바이,204473782,1658
175,2019-04,800,생활/문구/가구/식품,571584019,22497
176,2019-04,900,도서/티켓/취미/애완,1045449317,32317
177,2019-04,910,스타굿즈,655608810,26975
178,2019-04,999,기타,335689871,7788


In [89]:
# 2-Depth Category

In [90]:
df_bunp_cate_6 = pd.merge(df_bunp, df_category[['category', 'name']], 
                        how='left', left_on=['category_6'], right_on=['category'])

df_bunp_cate_6['month_at'] = df_bunp_cate_6['updated_at'].str[:7]

df_bunp_cate_6 = df_bunp_cate_6.rename(columns={'category_y': 'category', 'name_y': 'category_name'})

df_bunp_cate_6 = df_bunp_cate_6.groupby(['month_at',
                                     'category',
                                     'category_name'], as_index=False).agg({'bunp_amount': 'sum',
                                                           'selled': 'count'})

df_bunp_cate_6.tail()

,month_at,category,category_name,bunp_amount,selled
1405,2019-04,910400,솔로(여),3402100,140
1406,2019-04,910500,솔로(남),4491000,189
1407,2019-04,910600,배우(남),3760934,115
1408,2019-04,910700,배우(여),53000,6
1409,2019-04,910800,기타(방송인),2756138,131


## 번프와 등록된 상품 합치기

In [91]:
# 1-Depth Category

In [104]:
df_bunp_enroll_3 = pd.merge(df_bunp_cate_3, df_on_month_3, on=['month_at', 'category'])

# save
save_path = '../csv/tmp/df_bunp_enroll_3.csv'
df_bunp_enroll_3.to_csv(save_path, index=False, mode='w', header=True)

df_bunp_enroll_3.tail()

,month_at,category,category_name,bunp_amount,selled,enrolled
174,2019-04,750,차량/오토바이,204473782,1658,61796
175,2019-04,800,생활/문구/가구/식품,571584019,22497,59872
176,2019-04,900,도서/티켓/취미/애완,1045449317,32317,107936
177,2019-04,910,스타굿즈,655608810,26975,62164
178,2019-04,999,기타,335689871,7788,40951


In [93]:
# 2-Depth Category

In [110]:
df_bunp_enroll_6 = pd.merge(df_bunp_cate_6, df_on_month_6, on=['month_at', 'category'])

# save
save_path = '../csv/tmp/df_bunp_enroll_6.csv'
df_bunp_enroll_6.to_csv(save_path, index=False, mode='w', header=True)

df_bunp_enroll_6.tail()

,month_at,category,category_name,bunp_amount,selled,enrolled
1403,2019-04,910400,솔로(여),3402100,140,273
1404,2019-04,910500,솔로(남),4491000,189,591
1405,2019-04,910600,배우(남),3760934,115,281
1406,2019-04,910700,배우(여),53000,6,35
1407,2019-04,910800,기타(방송인),2756138,131,303


## 키워드 검색 - 보류

In [13]:
# 키워드 검색 수 by date
read_path = '../csv/keywords_1901_1904.csv'
df_keyword_1901_1904 = pd.read_csv(read_path)

read_path = '../csv/keywords_1807_1812.csv'
df_keyword_1807_1812 = pd.read_csv(read_path)

df_keyword = pd.concat([df_keyword_1901_1904, df_keyword_1807_1812])

# save
save_path = '../csv/tmp/df_keyword.csv'
df_keyword.to_csv(save_path, index=False, mode='w', header=True)

df_keyword.tail()

,date_at,keyword,search_num
81942714,2018-12-31,ìœ„ì¸ì „,1
81942715,2018-12-31,실리콘케이스 스카이블루,1
81942716,2018-12-31,오프화이트 갤럭시 후드집업,1
81942717,2018-12-31,후방카메라cctv,2
81942718,2018-12-31,패딩오버트라우져,1


In [4]:
read_path = '../csv/tmp/df_keyword.csv'
df_keyword = pd.read_csv(read_path)

df_keyword.tail()

,date_at,keyword,search_num
141683920,2018-12-31,ìœ„ì¸ì „,1
141683921,2018-12-31,실리콘케이스 스카이블루,1
141683922,2018-12-31,오프화이트 갤럭시 후드집업,1
141683923,2018-12-31,후방카메라cctv,2
141683924,2018-12-31,패딩오버트라우져,1


In [5]:
read_path = '../csv/keyword_category_detail.csv'
df_category = pd.read_csv(read_path).drop(['_count', '_num'], axis=1)

df_category = df_category.rename(columns={'name': 'category_name'})

df_category.tail()


,keyword,category,category_name
1891622,💖신상🌸,320030100.0,맨투맨 티셔츠
1891623,💬찢청,320130200.0,청/데님 반바지
1891624,💯발렌시아가 빌 탑,400010100.0,숄더백
1891625,🚛정품 MCM,400083200.0,남성 벨트
1891626,🤘구찌 지갑,400060400.0,남자 중/반지갑


In [ ]:
df_key_cate = pd.merge(df_keyword, df_category[['keyword', 'category']],
                       how = 'left', on = ['keyword'])

df_key_cate.tail()